# Notebook for visualizing the created grid

In [83]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder (e.g., 'masteroppgave') by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Find and append 'masteroppgave' to sys.path dynamically
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

Project root found: /Users/thomasnitsche/Developer/skole/master/code/masteroppgave


In [85]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from schemes.lsh_grid import GridLSH

In [86]:
Grid = GridLSH(
        "G1",
        min_lat=41.14,
        max_lat=41.19,
        min_lon=-8.66,
        max_lon=-8.57,
        resolution=0.25,
        layers=5,
        meta_file="meta.txt",
        data_path="/data",
    )
print(Grid)

Grid: G1
Covering: (5.559754011677007, 7.536733782089804) km 
Resolution: 0.25 km 
Distortion: [0.1189828711294954, 0.155629560281487, 0.12925474997706543, 0.05161096815079891, 0.2310794404649125] km 
Dimensions: (22, 30) cells


In [90]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random

def visualize_grid(grid_lsh):
    """
    Visualizes the grid of the GridLSH object using folium.
    
    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    
    Returns:
    - A folium map object.
    """
    
    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each layer
        layer_group = FeatureGroupSubGroup(base_layer, name=f"Layer {layer_index + 1}")
        map_grid.add_child(layer_group)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(layer_group)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid

# Example usage
grid_map = visualize_grid(Grid)
grid_map.save("grid_visualization.html")
